In [1]:
%%capture
%matplotlib inline
# may take a minute to build matplotlib font cache

In [2]:
import radd
from radd import build

In [3]:
# read data into pandas DataFrame (http://pandas.pydata.org/)
# example_data contains data from 15 subjects in the 
# Reactive Stop-Signal task discussed in Dunovan et al., (2015)
example_data = radd.load_example_data()
example_data.head()

,idx,Cond,ttype,choice,response,acc,rt,ssd
0,28,bsl,go,go,1,1,0.59853,1000
1,28,bsl,go,go,1,1,0.52017,1000
2,28,bsl,go,go,1,1,0.54513,1000
3,28,bsl,go,go,1,1,0.57159,1000
4,28,bsl,go,go,1,1,0.50522,1000


# Formatting data for radd
## Required columns
* **idx**: Subject ID number
* **ttype**: Trial-Type ('go' if no stop-signal, 'stop' if stop-signal trial)
* **response**: Trial Response (1 if response recorded, 0 if no response)
* **acc**: Choice Accuracy (1 if correct, 0 if error)
* **rt**: Choice Response-Time (in seconds, can be any value no-response trials)
* **ssd**: Stop-Signal Delay (in milliseconds, 1000 on go trials)

## Optional columns
* input dataframe can contain columns for experimental conditions of interest (choose any name)
* in the dataframe above, the **Cond** column contains **'bsl'** and **'pnl'**  
    * in the **'bsl'** or **"Baseline"** condition, errors on **go** and **stop** trials are equally penalized 
    * in the  **'pnl'** or **"Caution"** condition, penalties are doubled for **stop** trial errors  (e.g., response=1)
* See below for fitting models with conditional parameter dependencies 
    * e.g., drift-rate depends on levels of 'Cond'

# Building a model

In [12]:
# initiate dependent process model 
model = build.Model(kind='xdpm', data=example_data)

# Animation of Dependent Process Model

In [15]:
# Initial state of Stop process (red) depends on current strength of Go activation (green)
# Assumes Stop signal efficacy at later SSDs diminishes as the state of the Go process 
# approaches the execution threshold (upper bound). pink lines denote t=SSD, blue is trial deadline
radd.load_dpm_animation()

# Header of observed dataframe (model.observedDF)
* **idx**: subject ID
* **Cond**: Baseline(bsl)/Caution(pnl) (could be any experimental condition of interest) 
* **Acc**: Accuracy on "go" trials
* **sacc**: Mean accuracy on "stop" trials (mean condition SSD used during simulations)
* **c10 - c90**: 10th - 90th RT quantiles for correct responses
* **e10 - e90**: 10th - 90th RT quantiles for error responses

In [14]:
model.observedDF.head()

,idx,flat,acc,200,250,300,350,400,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
0,28,flat,0.98347,1.0,1.0,0.95,0.700,0.050,0.50522,0.54506,0.55847,0.58503,0.61167,0.50518,0.53177,0.54524,0.55854,0.58443
1,29,flat,0.97934,1.0,1.0,1.00,0.825,0.175,0.53145,0.54670,0.57174,0.58526,0.62468,0.53145,0.54505,0.55842,0.57177,0.59776
2,30,flat,0.90496,1.0,1.0,1.00,0.725,0.350,0.51852,0.57165,0.58527,0.61174,0.63841,0.53180,0.55846,0.57194,0.59840,0.61176
3,31,flat,0.98347,1.0,1.0,1.00,0.750,0.150,0.53173,0.54522,0.55867,0.58501,0.61174,0.51605,0.54052,0.54576,0.57138,0.59847
4,32,flat,0.95868,1.0,1.0,1.00,0.750,0.100,0.54493,0.57169,0.58517,0.59855,0.63823,0.51842,0.54519,0.56475,0.58506,0.60014


# Bounded Global & Local Optimization


## Stochastic Global Optimization (Basinhopping w/ bounds)

**set_basinparams()** method gives control over low-level parameters used for global opt


* **xtol = ftol = tol**: error tolerance of global optimization (default=1e-20)


* **stepsize** (default=.05): set basinhopping initial step-size
    * see HopStep class in radd.fit for more details
    * see get_stepsize_scalars() in radd.theta for parameter-specific step-sizes


* **nsamples** (default=5000): number of parameter sets to sample
    * optimization is performed on best **ninits** (see below) 


* **ninits** (default=5): number of initial parameter sets to perform global optimization on
    * evaluate all **nsamples** parameter sets, select **ninits** with the lowest error 
    * perform global optimization using each parameter set (out of the **ninits** chosen)
    * pass the optimized parameters with the lowest global minimum to local optimizer 


* **nsuccess** (default=40): criterion number of successful steps without finding new global minimum to exit basinhopping


* **interval** (default=10): number of steps before adaptively updating the stepsize 


* **T** (default=1.0): set the basinhopping "temperature"
    * higher T will result in accepted steps with larger changes in function value (larger changes in model error)

## Local Optimization (Nelder-Mead Simplex w/ bounds)
**set_fitparams()** method gives control over low-level parameters used for local opt. Local optimization polishes parameter estimates passed from global optimization step.


* **xtol = ftol = tol** (default=1e-30): error tolerance of optimization


* **maxfev** (default=2000): max number of func evaluations to perform


* **ntrials** (default=20000): num. simulated trials per condition


* **quantiles** (default=array([.1, .3, .5, .7, .9])): quantiles of RT distribution to fit

In [16]:
# example of how to modify basinparams
model.set_basinparams(tol=1e-25, stepsize=.05, nsamples=2000, ninits=3)
# example of how to modify fitparams
model.set_fitparams(tol=1e-25, ntrials=10000, )

# Steps in Fitting Routine


## Step 1. Flat Fits
- All models are initially fit by optimizing the full set of parameters to the "flattened" data (flat meaning the average data collapsing across all conditions of interest). For instance, at this stage fitting the dependent process model involves finding the parameter values for each included parameter that minimizes the cost-function cost function: 

  $Cost = \sum [\omega * (\hat{Y} - Y)]^2$
  
  
- **$Y$** is an array of observed data (e.g., accuracy, RT quantiles, etc.) 


- **$\hat{Y}$** is an equal length array of corresponding model-predicted values given parameters $\theta$


- The error between the predicted and the observed data (**$\hat{Y} - Y$**) is weighted by an equal length array of scalars **$\omega$** proportional to the inverse of the variance in each value of **$Y$**. 


- The array of weighted differences is summed, yielding a single cost value equal to the summed squared-error (**$SSE$**). 


| Parameters ($\theta$) | Description | Go/Stop |
|:--:|:--|:---:|
| a | Boundary Height | -- |
| tr | Onset-Delay | Go |
| v | Drift-Rate | Go | 
| xb | Dynamic Gain | Go |
| ssv | Drift-Rate | Stop |
| sso | Onset-Delay | Stop |

### Step 1a: 
- Global optimzation on flat data (average values collapsing across experimental conditions)

### Step 1b:
- Local optimzation using parameters passed from global optimizer as starting values 


## Step 2. Conditional Fits
- Conditional models can be fit in which all parameters from **Step 1** are held constant except for one or more designated **conditional** parameters which is free to vary across levels an experimental condition of interest. Global and local optimization are performed by minimizing the cost-function: 

    $Cost = \sum_{i=0}^{N_c} [\omega_i * (\hat{Y_i} - Y_i)]^2$ 
  
  
  

- where $\sum[\omega_i*(\hat{Y_i} - Y_i)]^2$ gives the **$Cost$** for level **$i$** of condition **$C$** 


- the total **$Cost$** is equal to the **$SSE$** across all **$N_c$** levels of that condition


- Specifying parameter dependencies is done using **depends_on** --> **{parameter_id : condition_name}**.


- For instance, in Dunovan et al., (2015) subjects performed two versions of a stop-signal task 
    - **Baseline ("bsl")**: subjects received equal penalties for not responding on Go trials and executing a response on "Stop-Signal" trials
    - **Penalty ("pnl")**: responding on "Stop-Signal" trials elicits double the penalty for ommitted responses on "Go" trials
    - Different hypotheses about the mechanism underlying behavioral adaptation to assymetric risk of committing different types of errors

- To test the hypothesis that a change in decision threshold **(a)** provides a better account of the data than a change in the drift-rate **(v)**, where **'Cond'** is the name of a column header in your data file 
  
  
  | How-To |Code |
  |:---:|:---:|
  | initiate boundary model |**model_a = build.Model(data, depends_on={'a':'Cond'})** |
  | initiate drift model  | **model_v = build.Model(data, depends_on={'v':'Cond'})** |
  | fit boundary model | **model_a.optimize()** |
  | fit drift model | **model_v.optimize()** |
  | Is boundary model better? | **model_a.finfo['AIC'] < model_v.finfo['AIC']** |


### Step 2a
- Global optimzation of conditional parameters 

### Step 2b:
- Local optimzation of conditional parameters passed from global optimizer

# Boundary Model

In [39]:
# set progress=True to track the model error at each basinhopping step (red)
# as well as the global minimum (green) across all initial param sets 
model_a = build.Model(kind='xdpm', data=example_data, depends_on={'a':'Cond'}, fit_on='average')
model_a.optimize(progress=True)

# Drift Model

In [38]:
model_v = build.Model(kind='xdpm', data=example_data, depends_on={'v':'Cond'}, fit_on='average')
model_v.optimize(progress=True)

# Model Comparison

In [19]:
models = ['Boundary', 'Drift-Rate']
gof_names = ['AIC', 'BIC']

# extract GOF stats from finfo attribute
a_gof = model_a.finfo[gof_names]
v_gof = model_v.finfo[gof_names]

# print GOF stats for both models
print('Boundary GOF:\nAIC = {}\nBIC = {}\n'.format(*a_gof))
print('Drift-Rate GOF:\nAIC = {}\nBIC = {}\n'.format(*v_gof))

# Which model provides a better fit to the data?
aicwinner = models[np.argmin([a_gof[0], v_gof[0]])]
bicwinner = models[np.argmin([a_gof[1], v_gof[1]])]
print('AIC likes {} model'.format(aicwinner))
print('BIC likes {} model'.format(bicwinner))

Boundary GOF:
AIC = -264.681635456
BIC = -264.522752372

Drift-Rate GOF:
AIC = -273.288594222
BIC = -273.129711138

AIC likes Drift-Rate model
BIC likes Drift-Rate model


# Troubleshooting Ugly Fits

## Fit to individual subjects
* model = build.Model(data=data, ..., **fit_on**=**'subjects'**)

## Other "kinds" of models...
* Currently only Dependent Process Model **(kind='dpm')** and Independent Race Model **(kind='irace')**
* Tell model to include a Dynamic Bias Signal **('xb')** by adding an **'x'** to the front of model **kind**
* To implement the Dependent Process Model **('irace')**
    * with dynamic bias: model = build.Model(data=data, **kind**=**xdpm'** ... )
    * without: model = build.Model(data=data, **kind**=**dpm'**, ... )
* To implement the Independent Race Model **('irace')**
    * with dynamic bias: model = build.Model(data=data, **kind**=**xirace'** ... )
    * without: model = build.Model(data=data, **kind**=**irace'**, ... )


## Other dependencies...
* Maybe subjects change their boundary height or go onset time across conditions
* Model with dynamic gain free across conditions:
    * model = build.Model(data=data, ... , **depends_on**={**'xb'**: **'Cond'**})
* Model with onset-delay free across conditions:
    * model = build.Model(data=data, ... , **depends_on**={**'tr'**: **'Cond'**})
        

## Optimization parameters...
 * model.set_basinparams(nsuccess=50, tol=1e-30, ninits=10, nsamples=10000) 
 * model.set_fitparams(maxfev=5000, tol=1e-35)
 * Check out the wts vectors for extreme vals
     * Try re-running the fits with an unweighted model (all wts = 1) 
         * m = build.Model(data=data, ... weighted=False)
 * Error RTs can be particularly troublesome, sometimes un-shootably so...


## How to access...

|model information | method used to calculate | how to access|
|--|--|--|
| flat data | **model**.observedDF.mean() | **model**.observed_flat |
| flat weights | **model**.wtsDF.mean() | **model**.flat_wts |
| conditional data | **model**.observedDF.groupby(**condition**).mean()| **model**.observed |
| conditional weights | **model**.wtsDF.groupby(**condition**).mean() |  **model**.cond_wts |

# Examine fits

In [40]:
# the fit summary (goodness of fit measures, etc.) 
model_v.fitDF

idx,average
a,0.72542
ssv,-1.3773
tr,0.068631
xb,2.3725
v_bsl,0.82666
v_pnl,0.78942
nfev,1534
nvary,2
df,30
chi,0.0051742


In [42]:
# model predictions
# to save as csv file: model_v.yhatDF.to_csv("save_path", index=False)
# to extract values as numpy ndarray: model_v.yhatDF.loc[:, 'acc':].values
model_v.yhatDF.head()

,idx,Cond,acc,200,250,300,350,400,c10,c30,c50,c70,c90,e10,e30,e50,e70,e90
0,average,bsl,0.98915,1.0,1.0,0.9480,0.5160,0.105,0.51863,0.54363,0.55863,0.57863,0.60863,0.50863,0.53363,0.54363,0.56363,0.58363
1,average,pnl,0.97155,1.0,1.0,0.9685,0.6055,0.152,0.52363,0.55363,0.56863,0.58863,0.61363,0.51863,0.53863,0.55363,0.56863,0.59363


In [41]:
# best-fit parameter estimates also stored in popt dictionary
model_v.popt

{'a': 0.72542390377886112,
 'ssv': -1.3772911501517369,
 'tr': 0.068630687082771591,
 'v': array([ 0.82666,  0.78942]),
 'v_bsl': 0.82666378486669079,
 'v_pnl': 0.78942365261507419,
 'xb': 2.3724828585333579}

In [11]:
import radd
radd.style_notebook()

Notebook Theme: Grade3
more at github.com/dunovank/jupyter-themes
